# Imports

In [2]:
print('Loading libraries')
suppressPackageStartupMessages({
    library(data.table)
    library(ggplot2)
    library(ggpubr)
    library(dplyr)
    library(stringr)
    library(rstatix)
    library(tidyr)
    }
)
print('Library load finished')

hpc =  TRUE
work_desktop = FALSE
work_linux = TRUE
singularity = TRUE
prj =  "HCC-CBS-173-Hillman-BFerris-NRG-HN003-Vectra"
if (!hpc){
    if (work_desktop){
        if (work_linux){
            base = paste0('/mnt/d/OneDrive - University of Pittsburgh/Internal_Project_Data_2/' ,prj)
        }else{
            base =paste0('D:\\OneDrive - University of Pittsburgh\\Internal_Project_Data_2\\', prj)
        }
    }else{
        base = paste0('/home/brian/data/odrive/Internal_Project_Data_2/',prj)
    }
}else{
    if (singularity){
        base = '/mnt'
    }else{
        base = paste0('/ix/rbao/Projects/', prj)
    }
}
print(base)
inform.data <- file.path(base,'data','inform_data_exports')
results <- file.path(base,'results')
print(results)


[1] "Loading libraries"
[1] "Library load finished"
[1] "/mnt"
[1] "/mnt/results"


## Load data

In [134]:
fn <- 'NRG-HN003-7552156cell-measurements_11col.csv'
start <- Sys.time()
df <- read.csv(file.path(results,fn))
df$pdl1.cell <- 'other'
df$pdl1.cell[df$is.pdl1.cell] = 'all.pdl1.cell'

df$panck.pdl1.cell <-'other'
df$panck.pdl1.cell[df$is.panck.cell & df$is.pdl1.cell] = 'panck.pdl1.cell'
idx <- !((df$combined.pheno %like% 'CK+') | (df$combined.pheno %like% 'CD3+'))
df$panck.pdl1.cell[idx & df$is.pdl1.cell] <- 'panckneg.cd3neg.pdl1.cell'

stop <- Sys.time()
print(paste((stop-start),'seconds'))
head(df)

[1] "48.0321016311646 seconds"


,X,Sample.Name,fn,Cell.ID,cell.type,all.tumor.stroma,tissue.compartment,pdl1.expression,pd1.expression,combined.pheno,is.pdl1.cell,is.panck.cell,pdl1.cell,panck.pdl1.cell
,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<lgl>,<lgl>,<chr>,<chr>
1,1,RACTIH_Scan1.qptiff,"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",39799,other,Tumor,inner.tumor.inv.margin,0.125,0.316,other,FALSE,FALSE,other,other
2,2,RACTIH_Scan1.qptiff,"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",39800,panck,Tumor,central.tumor,0.360,0.107,CK+,FALSE,TRUE,other,other
3,3,RACTIH_Scan1.qptiff,"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",39801,other,Tumor,central.tumor,0.220,0.172,other,FALSE,FALSE,other,other
4,4,RACTIH_Scan1.qptiff,"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",39802,panck,Tumor,central.tumor,0.397,0.157,CK+,FALSE,TRUE,other,other
5,5,RACTIH_Scan1.qptiff,"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",39803,other,Tumor,central.tumor,0.111,0.235,PD1+,FALSE,FALSE,other,other
6,6,RACTIH_Scan1.qptiff,"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",39804,other,Tumor,central.tumor,0.420,0.154,other,FALSE,FALSE,other,other


In [135]:
df$pdl1.cell <- 'other'
df$pdl1.cell[df$is.pdl1.cell] = 'all.pdl1.cell'

df$panck.pdl1.cell <-'other'
df$panck.pdl1.cell[df$is.panck.cell & df$is.pdl1.cell] = 'panck.pdl1.cell'
idx <- !((df$combined.pheno %like% 'CK+') | (df$combined.pheno %like% 'CD3+'))
df$panck.pdl1.cell[idx & df$is.pdl1.cell] <- 'panckneg.cd3neg.pdl1.cell'

df %>% count(panck.pdl1.cell)

panck.pdl1.cell,n
<chr>,<int>
other,5478651
panck.pdl1.cell,1123403
panckneg.cd3neg.pdl1.cell,950102


# Subset data and debug necessary table operations

In [136]:
idx <- df$Sample.Name %in% c('RACTIH_Scan1.qptiff','RACTKE_Scan1.qptiff')
sub <- df[idx,]
dim(sub)

[1] 1154004      14

In [6]:
colnames(sub)

[1] "X"                  "Sample.Name"        "fn"                
 [4] "Cell.ID"            "cell.type"          "all.tumor.stroma"  
 [7] "tissue.compartment" "pdl1.expression"    "pd1.expression"    
[10] "combined.pheno"     "is.pdl1.cell"       "is.panck.cell"

In [60]:
sub %>% 
    group_by(Sample.Name, tissue.compartment, cell.type) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             cell.type = unique(cell.type), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name, tissue.compartment) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) 

`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.


Sample.Name,tissue.compartment,cell.type,n,percent_density
<chr>,<chr>,<chr>,<int>,<dbl>
RACTIH_Scan1.qptiff,central.tumor,cd8.t,4,0.0354
RACTIH_Scan1.qptiff,central.tumor,other,7191,63.5640
RACTIH_Scan1.qptiff,central.tumor,panck,4118,36.4006
RACTIH_Scan1.qptiff,central.tumor,treg,0,0.0000
RACTIH_Scan1.qptiff,distal.stroma,cd8.t,0,0.0000
RACTIH_Scan1.qptiff,distal.stroma,other,3062,91.5396
RACTIH_Scan1.qptiff,distal.stroma,panck,283,8.4604
RACTIH_Scan1.qptiff,distal.stroma,treg,0,0.0000
RACTIH_Scan1.qptiff,inner.tumor.inv.margin,cd8.t,3,0.0133


# Example for pdl1 expression

In [125]:
sub %>% 
    group_by(Sample.Name, tissue.compartment) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             fill = list(median_pdl1_expression = 0)) %>%    
    pivot_wider(names_from = c("tissue.compartment"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_pdl1.expression'),everything())
    # group_by(Sample.Name, tissue.compartment) %>%
    # mutate(percent_density = round(n / sum(n) * 100, digits=4)) 

`summarise()` has grouped output by 'Sample.Name'. You can override using the
`.groups` argument.


,inner.tumor.inv.margin_pdl1.expression,outer.tumor.inv.margin_pdl1.expression
,<dbl>,<dbl>
RACTIH_Scan1.qptiff,0.557,0.6635
RACTKE_Scan1.qptiff,4.744,4.0880


In [64]:
sub %>% 
    group_by(Sample.Name, tissue.compartment, cell.type) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             cell.type = unique(cell.type), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma, cell.type) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.tumor.stroma", "cell.type"),
               values_from = "percent_density") %>%
    select(-matches("other"))
    

`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.all.tumor.stroma'.
You can override using the `.groups` argument.


Sample.Name,all.stroma_cd8.t,all.stroma_panck,all.stroma_treg,all.tumor_cd8.t,all.tumor_panck,all.tumor_treg
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RACTIH_Scan1.qptiff,0.0092,9.8999,0.0046,0.0206,44.0977,0.0000
RACTKE_Scan1.qptiff,1.5515,24.6287,3.0400,0.0413,80.8149,0.0724


In [179]:
###################### # Calculate values for all tumor / stroma
#TREG CD8T
t.dense.all.ts <- df %>%  group_by(Sample.Name, tissue.compartment, cell.type) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             cell.type = unique(cell.type), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma, cell.type) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.tumor.stroma", "cell.type"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% select(-matches('panck')) %>%
    tibble::column_to_rownames(var = "Sample.Name")

#PANCK+/- PDL1
panck.pdl1.dense.all.ts <- df %>%  group_by(Sample.Name, tissue.compartment, panck.pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             panck.pdl1.cell = unique(panck.pdl1.cell), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma, panck.pdl1.cell) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.tumor.stroma", "panck.pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% 
    tibble::column_to_rownames(var = "Sample.Name")

# ALL PDL1
all.pdl1.dense.all.ts <- df %>%  group_by(Sample.Name, tissue.compartment, pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             pdl1.cell = unique(pdl1.cell), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma, pdl1.cell) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.tumor.stroma", "pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% select(-matches('panck')) %>%
    tibble::column_to_rownames(var = "Sample.Name")


###################### Calculate density in inner / outer tumor invasive margin
#TREG CD8T
t.dense.in.out <- df %>% 
    group_by(Sample.Name, tissue.compartment, cell.type) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             cell.type = unique(cell.type), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name, tissue.compartment) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.compartment", "cell.type"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name")

#ALL PDL1+
all.pdl1.dense.in.out <- df %>% group_by(Sample.Name, tissue.compartment, pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             pdl1.cell = unique(pdl1.cell), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name, tissue.compartment) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.compartment", "pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name")

#PANCK+/- PDL1+
panck.pdl1.dense.in.out <- df %>%  group_by(Sample.Name, tissue.compartment, panck.pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             panck.pdl1.cell = unique(panck.pdl1.cell), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name, tissue.compartment) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.compartment", "panck.pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>%
    tibble::column_to_rownames(var = "Sample.Name")

#################### # Calculate density for all tumor invasive margin
#TREG CD8T
t.dense.all.tiv <- df %>%  group_by(Sample.Name, tissue.compartment, cell.type) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             cell.type = unique(cell.type), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.invasive.margin = case_when(tissue.compartment %in% c('outer.tumor.inv.margin',
                                                                         'inner.tumor.inv.margin') ~ "all.tumor.invasive.margin",
                                                  TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.invasive.margin, cell.type) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.invasive.margin) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4)) %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.invasive.margin", "cell.type"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% select(-matches('panck')) %>%
    tibble::column_to_rownames(var = "Sample.Name")

# ALL PDL1
all.pdl1.dense.all.tiv <- df %>%  group_by(Sample.Name, tissue.compartment, pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             pdl1.cell = unique(pdl1.cell), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.invasive.margin = case_when(tissue.compartment %in% c('outer.tumor.inv.margin',
                                                                         'inner.tumor.inv.margin') ~ "all.tumor.invasive.margin",
                                                  TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.invasive.margin, pdl1.cell) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.invasive.margin) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4))  %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.invasive.margin", "pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% select(-matches('panck')) %>%
    tibble::column_to_rownames(var = "Sample.Name")

#PANCK+/- PDL1+
panck.pdl1.dense.all.tiv <- df %>%  group_by(Sample.Name, tissue.compartment, panck.pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             panck.pdl1.cell = unique(panck.pdl1.cell), 
             fill = list(n = 0)) %>%    
    mutate(tissue.all.invasive.margin = case_when(tissue.compartment %in% c('outer.tumor.inv.margin',
                                                                         'inner.tumor.inv.margin') ~ "all.tumor.invasive.margin",
                                                  TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.invasive.margin, panck.pdl1.cell) %>%
    summarize(n=sum(n)) %>%
    group_by(Sample.Name, tissue.all.invasive.margin) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4))  %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.all.invasive.margin", "panck.pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>% 
    tibble::column_to_rownames(var = "Sample.Name")

all.dense <- bind_cols(t.dense.all.ts, all.pdl1.dense.all.ts, panck.pdl1.dense.all.ts, 
                   t.dense.in.out, all.pdl1.dense.in.out, panck.pdl1.dense.in.out,
                   t.dense.all.tiv, all.pdl1.dense.all.tiv, panck.pdl1.dense.all.tiv) %>%
        rename_with(~paste0(.,'.density'),everything())
# head(all.dense)


#################################################################
#####################################################################
###################### PDL1 expression in tumor compartments

all.pdl1.exp.in.out <- df   %>%
    group_by(Sample.Name, tissue.compartment) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             fill = list(median_pdl1_expression = 0))  %>%    
    pivot_wider(names_from = c("tissue.compartment"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_all.pdl1.expression'),everything())

panck.pdl1.exp.in.out <- df   %>%
    group_by(Sample.Name, tissue.compartment,panck.pdl1.cell) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(median_pdl1_expression = 0))  %>%    
    pivot_wider(names_from = c("tissue.compartment","panck.pdl1.cell"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_pdl1.expression'),everything())

all.pdl1.exp.all.tiv <- df   %>% 
    mutate(tissue.all.invasive.margin = case_when(tissue.compartment %in% c('outer.tumor.inv.margin',
                                                                         'inner.tumor.inv.margin') ~ "all.tumor.invasive.margin",
                                                  TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.invasive.margin) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.all.invasive.margin = unique(tissue.all.invasive.margin), 
             fill = list(median_pdl1_expression = 0))  %>%   
    pivot_wider(names_from = c("tissue.all.invasive.margin"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_all.pdl1.expression'),everything())

panck.pdl1.exp.all.tiv <- df   %>% 
    mutate(tissue.all.invasive.margin = case_when(tissue.compartment %in% c('outer.tumor.inv.margin',
                                                                         'inner.tumor.inv.margin') ~ "all.tumor.invasive.margin",
                                                  TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.invasive.margin, panck.pdl1.cell) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.all.invasive.margin = unique(tissue.all.invasive.margin),
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(median_pdl1_expression = 0)) %>%   
    pivot_wider(names_from = c("tissue.all.invasive.margin","panck.pdl1.cell"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% 
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_pdl1.expression'),everything())

all.pdl1.exp.all.ts <- df   %>% 
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma ) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.all.tumor.stroma  = unique(tissue.all.tumor.stroma ), 
             fill = list(median_pdl1_expression = 0))  %>%   
    pivot_wider(names_from = c("tissue.all.tumor.stroma"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_all.pdl1.expression'),everything())

panck.pdl1.exp.all.ts <- df   %>% 
    mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma, panck.pdl1.cell ) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.all.tumor.stroma  = unique(tissue.all.tumor.stroma ),
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(median_pdl1_expression = 0))  %>%   
    pivot_wider(names_from = c("tissue.all.tumor.stroma", "panck.pdl1.cell"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_pdl1.expression'),everything())

panck.pdl1.exp.all.tissue <- df   %>% 
    group_by(Sample.Name, panck.pdl1.cell ) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(median_pdl1_expression = 0))  %>%   
    pivot_wider(names_from = c("panck.pdl1.cell"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>% 
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_pdl1.expression'),everything())

all.pdl1.exp.all.tissue <- df  %>%  
    group_by(Sample.Name) %>% 
    summarize(all.tissue=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             fill = list(all.tissue = 0))  %>%   
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'.all.cell_pdl1.expression'),everything())

panck.pdl1.exp.all.tissue <- df  %>%  
    group_by(Sample.Name, panck.pdl1.cell) %>% 
    summarize(all.tissue=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(all.tissue = 0))  %>%   
    pivot_wider(names_from = c("panck.pdl1.cell"),
               values_from = "all.tissue") %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    select(-matches("other")) %>%
    rename_with(~paste0('all.tissue.',.,'_pdl1.expression'),everything())

pdl1.expression <- bind_cols(all.pdl1.exp.all.tissue, panck.pdl1.exp.all.tissue,
                             all.pdl1.exp.all.tiv, panck.pdl1.exp.all.tiv,
                             all.pdl1.exp.in.out, panck.pdl1.exp.in.out,
                             all.pdl1.exp.all.ts, panck.pdl1.exp.all.ts,
                             )

all.measures <- bind_cols(all.dense, pdl1.expression)
t<-c()
for (nm in row.names(all.measures)){
    t <- c(t,str_split_fixed(nm,'_',n=2)[1])
}
row.names(all.measures) <- t
head(all.measures)
                   

`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.all.tumor.stroma'.
You can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.all.tumor.stroma'.
You can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.all.tumor.stroma'.
You can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.
`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.
`summarise()` has grouped

,all.stroma_cd8.t.density,all.stroma_treg.density,all.tumor_cd8.t.density,all.tumor_treg.density,all.stroma_all.pdl1.cell.density,all.tumor_all.pdl1.cell.density,all.stroma_panck.pdl1.cell.density,all.stroma_panckneg.cd3neg.pdl1.cell.density,all.tumor_panck.pdl1.cell.density,all.tumor_panckneg.cd3neg.pdl1.cell.density,⋯,inner.tumor.inv.margin_panck.pdl1.cell_pdl1.expression,inner.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panck.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression,all.stroma_all.pdl1.expression,all.tumor_all.pdl1.expression,all.stroma_panck.pdl1.cell_pdl1.expression,all.stroma_panckneg.cd3neg.pdl1.cell_pdl1.expression,all.tumor_panck.pdl1.cell_pdl1.expression,all.tumor_panckneg.cd3neg.pdl1.cell_pdl1.expression
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RACTIH,0.0092,0.0046,0.0206,0.0000,2.4819,1.5810,0.3137,2.1497,0.9645,0.6135,⋯,3.1975,3.3545,3.166,4.2080,0.592,0.503,3.0460,4.3280,3.2920,3.3545
RACTKE,1.5515,3.0400,0.0413,0.0724,36.9659,68.6116,21.3651,13.4975,56.0647,12.4281,⋯,6.1480,5.9830,7.295,7.4290,2.937,4.988,7.4730,7.5750,6.3260,6.2240
RACTLH,0.2723,0.1355,0.0031,0.0047,3.9697,8.7931,1.5995,2.2862,6.9951,1.7957,⋯,4.9440,4.7600,5.932,5.6060,0.513,0.802,5.8590,5.7180,4.9660,4.8925
RACTNY,1.2966,0.3303,0.3981,0.1524,0.9533,3.7857,0.1327,0.7500,2.8520,0.8977,⋯,1.1650,2.0900,1.479,2.7000,0.397,0.692,1.5375,4.1900,1.1230,2.1095
RACTRF,0.9067,2.0507,0.1553,0.8345,1.5253,14.2386,0.3559,1.0508,8.7204,5.3564,⋯,4.3580,4.2560,4.630,5.6930,0.500,1.396,4.7270,5.8645,4.5605,4.3845
RACTRU,1.7533,1.9068,2.0264,0.1981,8.9148,23.9201,0.7182,7.1438,11.5426,11.8491,⋯,4.9350,4.4630,5.460,6.0545,1.110,2.010,5.4175,6.2145,4.9890,4.5400


In [180]:
temp <- all.measures %>% select(matches('pdl1.expression'))
dim(temp)
print(colnames(temp))

[1] 34 18

 [1] "all.tissue.all.cell_pdl1.expression"                                
 [2] "all.tissuepanck.pdl1.cell_pdl1.expression"                          
 [3] "all.tissuepanckneg.cd3neg.pdl1.cell_pdl1.expression"                
 [4] "all.tumor.invasive.margin_all.pdl1.expression"                      
 [5] "all.tumor.invasive.margin_panck.pdl1.cell_pdl1.expression"          
 [6] "all.tumor.invasive.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression"
 [7] "inner.tumor.inv.margin_all.pdl1.expression"                         
 [8] "outer.tumor.inv.margin_all.pdl1.expression"                         
 [9] "inner.tumor.inv.margin_panck.pdl1.cell_pdl1.expression"             
[10] "inner.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression"   
[11] "outer.tumor.inv.margin_panck.pdl1.cell_pdl1.expression"             
[12] "outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression"   
[13] "all.stroma_all.pdl1.expression"                                     
[14] "all.tumor_all.pdl1.

In [163]:
row.names(all.measures) <- t
head(all.measures)

,all.stroma_cd8.t.density,all.stroma_treg.density,all.tumor_cd8.t.density,all.tumor_treg.density,all.stroma_all.pdl1.cell.density,all.tumor_all.pdl1.cell.density,all.stroma_panck.pdl1.cell.density,all.stroma_panckneg.cd3neg.pdl1.cell.density,all.tumor_panck.pdl1.cell.density,all.tumor_panckneg.cd3neg.pdl1.cell.density,⋯,inner.tumor.inv.margin_panck.pdl1.cell_pdl1.expression,inner.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panck.pdl1.cell_pdl1.expression,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression,all.stroma_all.pdl1.expression,all.tumor_all.pdl1.expression,all.stroma_panck.pdl1.cell_pdl1.expression,all.stroma_panckneg.cd3neg.pdl1.cell_pdl1.expression,all.tumor_panck.pdl1.cell_pdl1.expression,all.tumor_panckneg.cd3neg.pdl1.cell_pdl1.expression
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RACTIH,0.0092,0.0046,0.0206,0.0000,2.4819,1.5810,0.3137,2.1497,0.9645,0.6135,⋯,3.1975,3.3545,3.166,4.2080,0.592,0.503,3.0460,4.3280,3.2920,3.3545
RACTKE,1.5515,3.0400,0.0413,0.0724,36.9659,68.6116,21.3651,13.4975,56.0647,12.4281,⋯,6.1480,5.9830,7.295,7.4290,2.937,4.988,7.4730,7.5750,6.3260,6.2240
RACTLH,0.2723,0.1355,0.0031,0.0047,3.9697,8.7931,1.5995,2.2862,6.9951,1.7957,⋯,4.9440,4.7600,5.932,5.6060,0.513,0.802,5.8590,5.7180,4.9660,4.8925
RACTNY,1.2966,0.3303,0.3981,0.1524,0.9533,3.7857,0.1327,0.7500,2.8520,0.8977,⋯,1.1650,2.0900,1.479,2.7000,0.397,0.692,1.5375,4.1900,1.1230,2.1095
RACTRF,0.9067,2.0507,0.1553,0.8345,1.5253,14.2386,0.3559,1.0508,8.7204,5.3564,⋯,4.3580,4.2560,4.630,5.6930,0.500,1.396,4.7270,5.8645,4.5605,4.3845
RACTRU,1.7533,1.9068,2.0264,0.1981,8.9148,23.9201,0.7182,7.1438,11.5426,11.8491,⋯,4.9350,4.4630,5.460,6.0545,1.110,2.010,5.4175,6.2145,4.9890,4.5400


# PDL1 density

In [142]:
colnames(sub)

[1] "X"                  "Sample.Name"        "fn"                
 [4] "Cell.ID"            "cell.type"          "all.tumor.stroma"  
 [7] "tissue.compartment" "pdl1.expression"    "pd1.expression"    
[10] "combined.pheno"     "is.pdl1.cell"       "is.panck.cell"     
[13] "pdl1.cell"          "panck.pdl1.cell"

In [137]:
sub %>%   group_by(Sample.Name, tissue.compartment, panck.pdl1.cell) %>% 
    summarize(n=n()) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.compartment = unique(tissue.compartment), 
             panck.pdl1.cell = unique(panck.pdl1.cell), 
             fill = list(n = 0)) %>%    
    group_by(Sample.Name, tissue.compartment) %>%
    mutate(percent_density = round(n / sum(n) * 100, digits=4))  %>%
    select(-n) %>%
    pivot_wider(names_from = c("tissue.compartment", "panck.pdl1.cell"),
               values_from = "percent_density") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>%
    tibble::column_to_rownames(var = "Sample.Name")

`summarise()` has grouped output by 'Sample.Name', 'tissue.compartment'. You
can override using the `.groups` argument.


,inner.tumor.inv.margin_panck.pdl1.cell,inner.tumor.inv.margin_panckneg.cd3neg.pdl1.cell,outer.tumor.inv.margin_panck.pdl1.cell,outer.tumor.inv.margin_panckneg.cd3neg.pdl1.cell
,<dbl>,<dbl>,<dbl>,<dbl>
RACTIH_Scan1.qptiff,1.1333,0.6375,0.3655,2.3074
RACTKE_Scan1.qptiff,56.6102,9.1335,33.1450,14.5832


In [110]:
sub %>%     mutate(tissue.all.tumor.stroma = case_when(tissue.compartment %in% c("central.tumor",'inner.tumor.inv.margin') ~ "all.tumor",
                                tissue.compartment %in% c("distal.stroma",'outer.tumor.inv.margin') ~ "all.stroma",
                                TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.tumor.stroma ) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.all.tumor.stroma  = unique(tissue.all.tumor.stroma ), 
             fill = list(median_pdl1_expression = 0))  %>%   
    pivot_wider(names_from = c("tissue.all.tumor.stroma"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% select(-matches("panck")) %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_pdl1.expression'),everything())

`summarise()` has grouped output by 'Sample.Name'. You can override using the
`.groups` argument.


,all.stroma_pdl1.expression,all.tumor_pdl1.expression
,<dbl>,<dbl>
RACTIH_Scan1.qptiff,0.592,0.503
RACTKE_Scan1.qptiff,2.937,4.988


In [172]:
sub %>%  group_by(Sample.Name, panck.pdl1.cell) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(median_pdl1_expression = 0))  %>%   
    pivot_wider(names_from = c("panck.pdl1.cell"),
               values_from = "median_pdl1_expression") %>%
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    select(-matches("other")) %>%
    rename_with(~paste0(.,'_pdl1.expression'),everything())

`summarise()` has grouped output by 'Sample.Name'. You can override using the
`.groups` argument.


,panck.pdl1.cell_pdl1.expression,panckneg.cd3neg.pdl1.cell_pdl1.expression
,<dbl>,<dbl>
RACTIH_Scan1.qptiff,3.278,4.041
RACTKE_Scan1.qptiff,6.648,6.917


In [148]:
sub  %>%       mutate(tissue.all.invasive.margin = case_when(tissue.compartment %in% c('outer.tumor.inv.margin',
                                                                         'inner.tumor.inv.margin') ~ "all.tumor.invasive.margin",
                                                  TRUE ~ "other")) %>%
    group_by(Sample.Name, tissue.all.invasive.margin,panck.pdl1.cell) %>% 
    summarize(median_pdl1_expression=median(pdl1.expression)) %>%
    ungroup() %>%
    complete(Sample.Name = unique(Sample.Name), 
             tissue.all.invasive.margin = unique(tissue.all.invasive.margin),
             panck.pdl1.cell = unique(panck.pdl1.cell),
             fill = list(median_pdl1_expression = 0)) %>%   
    pivot_wider(names_from = c("tissue.all.invasive.margin","panck.pdl1.cell"),
               values_from = "median_pdl1_expression") %>%
    select(-matches("other")) %>%  select(-matches("distal")) %>%  
    select(-matches("central")) %>% 
    tibble::column_to_rownames(var = "Sample.Name") %>% 
    rename_with(~paste0(.,'_pdl1.expression'),everything())

`summarise()` has grouped output by 'Sample.Name',
'tissue.all.invasive.margin'. You can override using the `.groups` argument.


,all.tumor.invasive.margin_panck.pdl1.cell_pdl1.expression,all.tumor.invasive.margin_panckneg.cd3neg.pdl1.cell_pdl1.expression
,<dbl>,<dbl>
RACTIH_Scan1.qptiff,3.186,3.999
RACTKE_Scan1.qptiff,6.535,6.811


In [146]:
# Create a sample data.frame
df.test <- data.frame(x = 1:5)

# Rename the rows
df.test %>% 
  mutate(y = case_when(
    x == 1 ~ "one",
    x == 2 ~ "two",
    x == 3 ~ "three",
    TRUE ~ "other"
  ))

x,y
<int>,<chr>
1,one
2,two
3,three
4,other
5,other
